### Домашнее задание №3 по курсу «Графы и сети» (МТИИ, осень 2023)

Присылайте выполненное задание аналогично предыдущим домашним работам

In [1]:
!pip install pomegranate
import networkx as nx
import pomegranate as pg
import numpy as np
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 820.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 2.5 MB/s eta 0:00:00
  Created wheel for apricot-select: filename=apricot_select-0.6.1-py3-none-any.whl size=48764 sha256=82e43c5e4b5798b8b941c8a9f3e7d92d6431d92e49ea297f5a2246cdfeb5a785
  Stored in directory: /root/.cache/pip/wheels/df/d8/f9/4d62b7272bff772a126a52e507212c2fd33c0b8416d9389665
Successfully built apricot-select


### Задание 1
Постройте случайный граф на $n=200$ вершинах, согласно модели Эрдёша—Реньи с вероятностью появления ребра $p=\frac{1}{2}$. Напишите свой код генерации случайного графа, не пользуясь встроенным в `networkx` генератором [`gnp_random_graph`](https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.gnp_random_graph.html) и подобными. Посчитайте кликовое число в среднем по 10-20 сгенерированных графах. Насколько кликовое число далёко от теоретического среднего в $2\log_{1/p}n$?

In [2]:
# Ваш код:
np.random.seed(47)
rng = np.random.default_rng(seed=47)
n = 200
p = 0.5

def random_graph(vertices, probability):
    G = nx.Graph()
    G.add_nodes_from([node for node in range(vertices)])
    for i in range(vertices):
        for j in range(i+1, vertices):
            if rng.random() < probability:
                G.add_edge(i, j)

    return G


clique_numbers = []
for iter in range(10):
    G = random_graph(n, p)
    clique_numbers.append(max(len(c) for c in nx.find_cliques(G)))

np.mean(clique_numbers)

10.9

In [3]:
# Ваш код:
round(round(2 * np.log2(n), 1) - np.mean(clique_numbers), 1)

4.4

Ваши комментарии:

Среднее по $10$ сгенерированным графам кликовое число равно $10.9$, т.е. граф в среднем строится с кликами, из которых максимальная по размеру (по количеству вершин) клика построена на $10.9$ (очень часто клика строится на $11$ вершинах -- в $9$ из $10$ случаев) вершинах. Данное значение отличается от теоретического среднего на $4.4$, что весьма существенно. Наверное, оценка теоретического среднего предполагает $n \rightarrow \infty$ и дополнительный оттенок в формируемое значение кликового числа вносит стохастичность процесса, который не успевает стабилизироваться за $n=200$, но это не точно.

### Задание 2
Смоделируйте случайный граф в модели предпочтительного присоединения или другой отличной от Эрдёша—Реньи (согласно Вашему представлению о естественном росте, скажем, социальной сети). Опишите свою модель и приведите пример характеристик графа, которые отличаются в Вашей модели от соответствующих характеристик модели Эрдёша—Реньи.

In [13]:
a = set()
for i in range(10):
    a.add(i)
len(a)

10

In [18]:
# Ваш код:
np.random.seed(47)

n1 = 20
k = n1 // 5
n2 = 100
m = n2 // 20

def pref_attachment(n1, n2, k, m):
    g = nx.Graph()
    nodes = [node for node in range(n1)]
    for j in range(1, k//2 + 1):
        targets = nodes[j:] + nodes[0:j]
        g.add_edges_from([(nodes[i], targets[i]) for i in range(n1)])

    plt.figure(1)
    pos = nx.circular_layout(g)
    plt.figure(figsize=(20, 12))
    nx.draw(g, pos, node_size=60, font_size=8, with_labels=True)
    plt.show()

    G = g.copy()
    repeated_nodes = [n for n, d in G.degree() for _ in range(d)]
    source = len(G)
    while source < n2:
        targets = set()
        while len(targets) < m:
            x = np.random.choice(repeated_nodes)
            targets.add(x)
        G.add_edges_from(zip([source] * m, targets))
        # G.add_edges_from([(source, list(targets)[i]) for i in range(len(targets))])
        repeated_nodes.extend(targets)
        repeated_nodes.extend([source] * m)

        source += 1

    plt.figure(2)
    pos = nx.circular_layout(G)
    plt.figure(figsize=(20, 12))
    nx.draw(G, pos, node_size=60, font_size=8, with_labels=True)
    plt.show()

pref_attachment(n1, n2, k, m)

Output hidden; open in https://colab.research.google.com to view.

Ваши комментарии:

Для построения графа была использована модель Барабаши-Альберта, использующая правило предпочтительного связывания. Сначала был построен цикл на $n_{1}$ вершинах, у каждой из этих вершин задавалось $k$ ближайших соседей, между которыми (вершиной и её соседом) проводилось ребро. Затем из полученного графа, который будем называть графом-затравкой, выращивался новый граф на $n_{2}$ вершинах. Каждая вершина в таком графе присоединялась к уже существующим вершинам с вероятностью пропорциональной степени этих вершин -- принцип «богатый становится богаче» (предпочтительного связывания).

В качестве параметров модели задаются:


*   $n_{1}~-~$количество вершин в графе-затравке;
*   $n_{2}~-~$количество вершин в выращенном графе;
*   $k~-~$количество ближайших соседей каждой вершины графа, которое мы предполагаем равным по обеим сторонам графа (в представлении графа в виде круга или эллипса);
*   $m~-~$количество рёбер, которые можно пустить от нового узла к существующим узлам.

P.S. также на стадии выращивания графа целевые вершины, в которые испускаются рёбра, сэмплируются из заранее заданной выборки, в которой вершины могут повторяться в зависимости от степени данной вершины, в предположении равномерного распределения для всех экземпляров, в том числе и повторяющихся. Это означает, что с наибольшей вероятностью будем выбирать те вершины, степень которых максимальна. После добавления ребра обновляем степени затронутых вершин путём добавления этих вершин в выборку, из которой сэмплировали целевые вершины ранее, и продолжаем наращивание графа.

### Задание 3
Смоделируйте с помощью скрытой марковской модели следующую ситуацию. Гражданин Рассеянный в зависимости (почти) от погоды надевает либо футболку, либо плащ, либо тулуп. Когда жарко, он равновероятно надевает каждый из трёх видов одежды. Когда холодно, он не надевает плащ, а равновероятно надевает футболку либо тулуп. Когда температура средняя, он надевает исключительно тулуп либо плащ. Температура никогда не меняется резко (жаркая погода сменяется на холодную, и наоборот, — только через среднее состояние); при этом каждое состояние погоды равновероятно может остаться без изменений либо смениться на любое другое допустимое. На протяжении недели Рассеянный постил свои фото в Instagram в следующей одежде:
- пн: тулуп,
- вт: тулуп,
- ср: плащ,
- чт: футболка,
- пт: плащ,
- сб: плащ,
- вс: футболка.

Какая наиболее вероятная погода стояла на улице Бассейной в каждый из этих дней?

In [5]:
# Ваш код:
from pomegranate.distributions import Categorical
from pomegranate.hmm import DenseHMM

'''
Обозначения: ф -- футболка, п -- плащ, т -- тулуп
Индексация одежды: 0 -- футболка, 1 -- плащ, 2 -- тулуп
Индексация погоды: 0 -- холодно, 1 -- средняя температура, 2 -- жара
'''
dist_hot = Categorical([[1/3, 1/3, 1/3]])   # в соответствии с индексацией одежды

dist_norm = Categorical([[0, 1/2, 1/2]])    # в соответствии с индексацией одежды

dist_cold = Categorical([[1/2, 0, 1/2]])    # в соответствии с индексацией одежды

weather = DenseHMM()
weather.add_distributions([dist_cold, dist_norm, dist_hot])   # в соответствии с индексацией погоды
weather.start.name = 'start'
weather.end.name = 'end'

weather.add_edge(dist_hot, dist_norm, 0.5)
weather.add_edge(dist_hot, dist_hot, 0.5)
weather.add_edge(dist_cold, dist_norm, 0.5)
weather.add_edge(dist_cold, dist_cold, 0.5)
weather.add_edge(dist_norm, dist_norm, 0.5)
weather.add_edge(dist_norm, dist_hot, 0.25)
weather.add_edge(dist_norm, dist_cold, 0.25)

weather.add_edge(weather.start, dist_hot, 0.2)
weather.add_edge(weather.start, dist_norm, 0.4)
weather.add_edge(weather.start, dist_cold, 0.4)

weather.add_edge(dist_hot, weather.end, 0.5)
weather.add_edge(dist_cold, weather.end, 0.5)

seq = "ттпфппф"
X = np.array([[[['ф', 'п', 'т'].index(char)] for char in seq]])
preds = weather.predict(X)
weathers = []
for predictions in preds:
    for pred in predictions:
        if pred == 0:
            weathers.append('холодно')
        elif pred == 1:
            weathers.append('средняя температура')
        else:
            weathers.append('жара')

print(f'Sequence: {" ⟶ ".join(seq)}')
print(' ⟶ '.join(str(y.item()) for y in preds[0]))   # в соответствии с индексацией погоды
print(' ⟶ '.join(pred for pred in weathers))

Sequence: т ⟶ т ⟶ п ⟶ ф ⟶ п ⟶ п ⟶ ф
1 ⟶ 1 ⟶ 1 ⟶ 2 ⟶ 1 ⟶ 1 ⟶ 2
средняя температура ⟶ средняя температура ⟶ средняя температура ⟶ жара ⟶ средняя температура ⟶ средняя температура ⟶ жара
